In [13]:
import requests
from dotenv import load_dotenv
import os
load_dotenv()

GOCARDLESS_CLIENT_ID = os.getenv('GOCARDLESS_CLIENT_ID')
GOCARDLESS_CLIENT_SECRET = os.getenv('GOCARDLESS_CLIENT_SECRET')
if not GOCARDLESS_CLIENT_ID or not GOCARDLESS_CLIENT_SECRET:
    raise ValueError('GOCARDLESS_CLIENT_ID and GOCARDLESS_CLIENT_SECRET must be set')
else:
    print('GOCARDLESS_CLIENT_ID and GOCARDLESS_CLIENT_SECRET are set')


''' STEP 1 - ACCESS TOKEN '''
# GoCardless OAuth endpoint for obtaining access tokens
AUTH_URL = 'https://bankaccountdata.gocardless.com/api/v2/token/new/'


def get_access_token():
    # Data to be sent in the POST request
    data = {
        "secret_id": GOCARDLESS_CLIENT_ID,
        "secret_key": GOCARDLESS_CLIENT_SECRET
    }

    # Making the POST request to obtain the access token
    response = requests.post(AUTH_URL, json=data)
    response.raise_for_status()  # Raise exception for non-200 responses

    # Parsing the JSON response
    access_token = response.json()
    return access_token

access_token = get_access_token()['access']

''' END USER SELECTING A BANK '''
"https://github.com/nordigen/nordigen-bank-ui"

url = "https://bankaccountdata.gocardless.com/api/v2/institutions/"
params = {
    "country": "PT"
}
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {access_token}"  # Replace ACCESS_TOKEN with your actual access token
}

list_of_banks = requests.get(url, headers=headers, params=params)

if list_of_banks.status_code == 200:
    list_of_banks = list_of_banks.json()
    print("Response:")
    print(list_of_banks)
else:
    print("Error:", list_of_banks.text)
    list_of_banks.raise_for_status()  # Raise exception for non-200 responses


GOCARDLESS_CLIENT_ID and GOCARDLESS_CLIENT_SECRET are set
Response:
[{'id': 'ABANCA_CAGLPTPL', 'name': 'Abanca', 'bic': 'CAGLPTPL', 'transaction_total_days': '90', 'countries': ['PT'], 'logo': 'https://cdn-logos.gocardless.com/ais/ABANCA_CAGLESMM.png', 'max_access_valid_for_days': '180'}, {'id': 'ABANCA_CORP_CAGLPTPL', 'name': 'Abanca Corporate', 'bic': 'CAGLPTPL', 'transaction_total_days': '90', 'countries': ['PT'], 'logo': 'https://cdn-logos.gocardless.com/ais/ABANCA_CAGLESMM.png', 'max_access_valid_for_days': '180'}, {'id': 'AIRWALLEX_AIPTAU32', 'name': 'Airwallex', 'bic': 'AIPTAU32', 'transaction_total_days': '730', 'countries': ['DK', 'FI', 'GB', 'CY', 'LT', 'CZ', 'GR', 'PT', 'SI', 'EE', 'LU', 'FR', 'IE', 'PL', 'NO', 'LI', 'SE', 'SK', 'BG', 'HR', 'NL', 'AT', 'DE', 'BE', 'ES', 'LV', 'MT', 'RO', 'IT', 'IS', 'HU'], 'logo': 'https://cdn-logos.gocardless.com/ais/AIRWALLEX_AIPTAU32_1.png', 'max_access_valid_for_days': '90'}, {'id': 'BANCOACTIVOBANK_ACTVPTPL', 'name': 'Banco ActivoBank',

In [4]:
def extract_schema(json_list):
    """
    Extract schema from a list of dictionaries, showing all possible keys and their data types
    
    Args:
        json_list (list): List of dictionaries
    
    Returns:
        dict: Schema showing keys and their possible data types
    """
    schema = {}
    
    # Handle single dictionary case
    if isinstance(json_list, dict):
        json_list = [json_list]
    
    for item in json_list:
        for key, value in item.items():
            # Get the type of the value
            value_type = type(value).__name__
            
            # If key exists, add new type if different
            if key in schema:
                if value_type not in schema[key]:
                    schema[key].append(value_type)
            else:
                schema[key] = [value_type]
    
    return schema

# Example usage with your list_of_banks data:
schema = extract_schema(list_of_banks)
for key, types in schema.items():
    print(f"{key}: {types}")

id: ['str']
name: ['str']
bic: ['str']
transaction_total_days: ['str']
countries: ['list']
logo: ['str']
max_access_valid_for_days: ['str']


In [14]:
''' STEP 3 - CREATE END USER AGREEMENT '''
# API endpoint
url = "https://bankaccountdata.gocardless.com/api/v2/agreements/enduser/"

# Headers
headers = {
    "Authorization": f"Bearer {access_token}"
}

INSTITUTION_ID = "NATWEST_NWBKGB2L"
# Data
data = {
    "institution_id": INSTITUTION_ID,
    "max_historical_days": "730",
    "access_valid_for_days": "30",
    "access_scope": ["balances", "details", "transactions"]
}

# Make POST request
try:
    response = requests.post(url, json=data, headers=headers)
    response.raise_for_status()  # Raise exception for non-200 responses
    result = response.json()
    print(result)
except requests.exceptions.RequestException as e:
    print(f"Error making POST request: {e}")

''' STEP 4 - BUILD A LINK '''
import random
def generate_random_id():
    return ''.join(random.choices('0123456789', k=21))
random_id = generate_random_id()

url = "https://bankaccountdata.gocardless.com/api/v2/requisitions/"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": f"Bearer {access_token}"  # Replace ACCESS_TOKEN with your actual access token
}
data = {
    #"redirect": "https://michaelalilondon.bubbleapps.io/version-test",
    "redirect": "http://localhost:3001/gocardless/callback",
    "institution_id": result['institution_id'],
    "reference": random_id,
    "agreement": result['id'],
    "user_language": "EN"
}
response = requests.post(url, headers=headers, json=data)
response.json()

{'id': '3787f33d-5a66-4efe-bfb6-b091d97c8a54', 'created': '2025-02-12T17:24:09.400474Z', 'institution_id': 'NATWEST_NWBKGB2L', 'max_historical_days': 730, 'access_valid_for_days': 30, 'access_scope': ['balances', 'details', 'transactions'], 'accepted': None}


{'id': 'b37db951-fdd8-4b2e-ac0c-0b8c65f34598',
 'created': '2025-02-12T17:24:09.619843Z',
 'redirect': 'http://localhost:3001/gocardless/callback',
 'status': 'CR',
 'institution_id': 'NATWEST_NWBKGB2L',
 'agreement': '3787f33d-5a66-4efe-bfb6-b091d97c8a54',
 'reference': '974506665365154367098',
 'accounts': [],
 'user_language': 'EN',
 'link': 'https://ob.gocardless.com/ob-psd2/start/2039225d-0af2-409b-8ff4-ba6beb6c04a6/NATWEST_NWBKGB2L',
 'ssn': None,
 'account_selection': False,
 'redirect_immediate': False}

In [ ]:
# def extract_schema_nested(data, path=""):
#     """
#     Extract schema from JSON data, including nested structures and arrays
    
#     Args:
#         data: JSON data (dict, list, or primitive type)
#         path: Current path in the JSON structure (used for recursion)
    
#     Returns:
#         dict: Schema showing paths and their data types
#     """
#     schema = {}
    
#     if isinstance(data, dict):
#         for key, value in data.items():
#             current_path = f"{path}.{key}" if path else key
#             if isinstance(value, (dict, list)):
#                 schema.update(extract_schema_nested(value, current_path))
#             else:
#                 schema[current_path] = type(value).__name__
                
#     elif isinstance(data, list) and data:  # Non-empty list
#         # Handle first item in list as representative sample
#         sample_item = data[0]
#         if isinstance(sample_item, (dict, list)):
#             schema.update(extract_schema_nested(sample_item, f"{path}[]"))
#         else:
#             schema[f"{path}[]"] = type(sample_item).__name__
            
#     else:  # Primitive type
#         schema[path] = type(data).__name__
        
#     return schema

# # Example usage:
# response.json()
# schema = extract_schema_nested(response.json())
# for path, type_name in schema.items():
#     print(f"{path}: {type_name}")

In [ ]:
''' STEP 5 - LIST ACCOUNTS '''

url = f"https://bankaccountdata.gocardless.com/api/v2/requisitions/"

response = requests.get(url, headers=headers)
response.json()

### Get transaction data

In [5]:
requsitions = {
    "barclays": "881a82f5-9c82-4476-a88f-34921f0f5c9c",
    "natwest": "e1849ee0-6146-4cb8-8f16-a80c1eba6bca",
}

""" new account ids are issued with each new requisition """
natwest_accounts = [
    '81451d40-c0fb-4524-9ef0-7b36f7b2cc50']

barclays_accounts = [
    'd318a85e-58c1-4249-87a2-058bb817c070',
    'f9ee4378-ecc9-45c6-aedc-d982a7329072']

### accounts endpoint takes account id. 
url = f"https://bankaccountdata.gocardless.com/api/v2/accounts/{natwest_accounts[0]}/transactions/"

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {access_token}"  # Replace ACCESS_TOKEN with your actual access token
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    print("Response:")
    print(response.json())
else:
    print("Error:", response.text)


Response:
{'transactions': {'booked': [{'transactionId': '230931F0A990F8511EE8D54D98E35FDDCD22823A2BA87FEC4F9941360E377C2DCFF2974D03893C110BCD9D69F75ECABE', 'bookingDate': '2025-02-10', 'bookingDateTime': '2025-02-10T00:00:00.000Z', 'transactionAmount': {'amount': '-25.21', 'currency': 'GBP'}, 'creditorName': 'SUPABASE SINGAPORE SG', 'remittanceInformationUnstructured': '2876 07FEB25      SUPABASE          SINGAPORE SG      USD          30.33VRATE       1.2359N-S TRN FEE   0.67', 'proprietaryBankTransactionCode': 'POS', 'internalTransactionId': 'b6247e97e470df33e1c90fdefd2e07a3'}, {'transactionId': '230931F0A990F8511EE8D54D98E35FDDCD22823A2BA87FEC4F9941360E377C2D4644C2E2D44AA2F9682BDA3F04C0AE6A', 'bookingDate': '2025-02-10', 'bookingDateTime': '2025-02-10T00:00:00.000Z', 'transactionAmount': {'amount': '-49.90', 'currency': 'GBP'}, 'creditorName': 'CURSOR, AI POWEREDIDE NEW YORK US', 'remittanceInformationUnstructured': '2876 07FEB25      CURSOR, AI POWEREDIDE               NEW YORK US

In [8]:
import pandas as pd
from datetime import datetime

def get_transactions_for_account(account_id, access_token):
    """Fetch transactions for a single account"""
    url = f"https://bankaccountdata.gocardless.com/api/v2/accounts/{account_id}/transactions/"
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {access_token}"
    }
    
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()['transactions']['booked']

# # Process each bank's accounts
# for bank_name, accounts in {
#     'natwest': natwest_accounts,
#     'barclays': barclays_accounts
# }.items():
#     # Process each account for this bank
#     for account_id in accounts:
#         try:
#             # Get transactions
#             transactions = get_transactions_for_account(account_id, access_token)
            
#             # Convert to DataFrame
#             df = pd.DataFrame(transactions)
            
#             # Generate filename using bank name and last 3 chars of account_id
#             filename = f"{bank_name}_{account_id}.csv"
            
#             # Save to CSV
#             df.to_csv(filename, index=False)
#             print(f"Successfully saved {len(transactions)} transactions to {filename}")
            
#         except Exception as e:
#             print(f"Error processing account {account_id}: {str(e)}")

In [ ]:
transactions = get_transactions_for_account(natwest_accounts[0], access_token)

# Convert to DataFrame
df = pd.DataFrame(transactions)

# Generate filename using bank name and last 3 chars of account_id
filename = f"natwest_{natwest_accounts[0][:-3]}.csv"

# Save to CSV
df.to_csv(filename, index=False)
print(f"Successfully saved {len(transactions)} transactions to {filename}")



Successfully saved 117 transactions to natwest_81451d40-c0fb-4524-9ef0-7b36f7b2c.csv
